In [3]:
# from pauli import PauliSum, SiteBasis, PauliString
from openfermion import FermionOperator, QubitOperator, jordan_wigner, reverse_jordan_wigner
from openfermion.utils import commutator
from hamiltonians import transverse_field_ising_model
from pauli import PauliString, PauliSum, SiteBasis

import cvxpy as cp
import numpy as np
from scipy.sparse import csr_array
import matplotlib.pyplot as plt

In [11]:
qtfim = transverse_field_ising_model(2, 0)
ftfim = reverse_jordan_wigner(qtfim.to_qubit_operator())

In [ ]:
def beta_independent_constraints(basis: SiteBasis, B: cp.Variable, L: int, hamil: PauliSum) -> list[cp.Constraint]:
    constraints = [B[0] == 1.0]
    row_ind, col_ind, data = [], [], []
    n_rows = 0
    n_cols = B.shape[0]
    for word in basis._ops:
        psum = schwinger_dyson(hamil, word)
        if psum.is_zero():
            continue
        for pstr, weight in psum.terms.items():
            row_ind.append(n_rows)
            col_ind.append(basis.rank(str(pstr)))
            data.append(np.imag(weight))
        n_rows += 1
    if len(col_ind) > 0:
        P = csr_array((data, (row_ind, col_ind)), shape=(n_rows, n_cols))
        constraints.append(P @ B == 0)
    return constraints

def kms(basis: SiteBasis, B: cp.Variable, hamil: PauliSum, beta: cp.Parameter, k: int):
    ops = basis._ops
    n = len(ops)
    row_inds = [[] for _ in range(n)]
    col_inds = [[] for _ in range(n)]
    datas = [[] for _ in range(n)]
    for i in range(n):
        for j in range(n):
            pi = PauliString(ops[i])
            pj = PauliString(ops[j])
            pstr = pi * pj
            idx = basis.rank(str(pstr))
            row_inds[idx].append(i)
            col_inds[idx].append(j)
            datas[idx].append(pi.phase(pj))
    A = cp.sum([csr_array((datas[i], (row_inds[i], col_inds[i])), shape=(n, n)) * B[i] for i in range(n)])
    row_inds = [[] for _ in range(n)]
    col_inds = [[] for _ in range(n)]
    datas = [[] for _ in range(n)]
    for i in range(n):
        for j in range(n):
            pi = PauliSum(ops[i])
            pj = PauliSum(ops[j])
            psum = pi * (hamil | pj)
            if psum.is_zero():
                continue
            # Now, we have a PauliSum of terms...and also some zero locations
            # Experiments suggest that the sparsity is not much
            # Now each non-zero location is some linear combination of Pauli strings
            # in the reduced basis...how to deal with this?
            # Well, now it is a linear matrix inequality...just write as F[i] * x[i]
            # These F[i] are sparse
            for pstr, weight in psum.terms.items():
                idx = basis.rank(str(pstr))
                row_inds[idx].append(i)
                col_inds[idx].append(j)
                datas[idx].append(weight)
    C = cp.sum([csr_array((datas[i], (row_inds[i], col_inds[i])), shape=(n, n)) * B[i] for i in range(n)])
    T = 2**(-k) * beta * C
    Zs = [A.T] + [cp.Variable((n,n),f'Z_{i}',hermitian=True) for i in range(1, k + 1)]
    constraints = []
    for i in range(k):
        constraints.append(cp.bmat([[Zs[i], Zs[i+1]],
                                    [Zs[i+1], A]]) >> 0)
    al, be, ga, de = -1, 5, 2, 4
    b = (be*de - ga) / (be - al)
    a = de - b
    constraints.append(cp.bmat([[Zs[k] + al*A + a*T, np.sqrt(a*b) * T],
                                [np.sqrt(a*b) * T, Zs[k] + be*A + b*T]]) >> 0)
    return constraints